In [2]:
#import sys 
#reload(sys) 
#sys.setdefaultencoding('utf8')
import cPickle
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
import numerical_routines as nr
import collections 
import re
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix 

# Import a lot of models...
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import random

# Import Tree Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import VotingClassifier

# The size of the dictionary array vectors
dictionary_size = 3000
data_set_size = 500 # Take a subset of the data, max size = 23,723
split_frac = 0.8 # The parameter that sets the train/test set size

# Here we read in the data-set of real and fake news items
real_and_fake_df = pd.read_csv('data_file/fake_and_real_news_titles.csv')
real_and_fake_df.head()

# Take a subset of the data
real_and_fake_df = real_and_fake_df[0:data_set_size]


# Now we split the data into the training set and test data set
data_set_size = len(real_and_fake_df)
split_indx = int(data_set_size*split_frac)

training_df = real_and_fake_df[:split_indx]
testing_df = real_and_fake_df[split_indx:]

training_size = len(training_df)
testing_size = len(testing_df)

print 'Training Set Number: ', training_size
print 'Testing Set Number: ', testing_size

# Now the data from the story titles is extracted into one large array containing all words in the title
title_words_array = nr.aggregate_data(training_df.dropna(subset=['title']),size= training_size)
#print title_words_array
# Now we make filter the title words 
filtered_title_words_array = nr.create_dictionary(title_words_array)


# Now we count the frequency of the different words
dictionary = collections.Counter(filtered_title_words_array)

list_to_remove = dictionary.keys()

# Remove one character items 
for key in list_to_remove:
    
    if len(key) == 1:
        del dictionary[key]

# Now take a reasonable subset of the data
dictionary = dictionary.most_common(dictionary_size)

print '=============================================================='
print 'The Size of the dictionary', len(filtered_title_words_array)
print 'Top Dictionary Words: ', dictionary[0:10] 
print ''
print '=============================================================='

# We convert a data set into a feature matrix
feature_matrix = nr.create_featureMatrix(training_df,dictionary,size= training_size)

# We find the labels of the data that we converted into the feature matrix and convert into an array
label_array = nr.create_label_array(training_df,size= training_size)

print 'array of labels size: ', len(label_array)
print 'feature matrix: ', len(feature_matrix)

print '========================================='
print 'Feature Matrix computation completed'
print '========================================='



# Now we train the various classifier
model1 = MultinomialNB()
model2 = LinearSVC()

# Model
m_DecTree = DecisionTreeClassifier(max_depth=5)
m_RandForest = RandomForestClassifier(max_depth=5, n_estimators=50)
m_ExTree = ExtraTreesClassifier(max_depth=5, n_estimators=50)


print '========================================================================================================================'
print 'Fitting will now take place'
print '========================================================================================================================'


model1.fit(feature_matrix,label_array)
model2.fit(feature_matrix,label_array)
mixingWeights = [1.0,1.0,1.0]
numOfNeighs = 1 

#model3 = VotingClassifier(estimators=[ ('dtc', clf1), ('rfc', clf2), ('etc', clf3) ], voting='soft', weights= mixingWeights )
model3 = VotingClassifier(estimators=[ ('dtc', m_DecTree), ('rfc', m_RandForest), ('etc', m_ExTree) ], voting='soft', weights= mixingWeights )
model3.fit(feature_matrix, label_array)

# fit models individually
m_DecTree.fit(feature_matrix, label_array)
m_RandForest.fit(feature_matrix, label_array)
m_ExTree.fit(feature_matrix, label_array)

# More Models
#clf4 = AdaBoostClassifier(n_estimators=50)
#clf5 = GradientBoostingClassifier(n_estimators=50)
#clf6 = KNeighborsClassifier(n_neighbors=numOfNeighs)

# More Models
m_AdaBoost = AdaBoostClassifier(n_estimators=50)
m_gradBoost = GradientBoostingClassifier(n_estimators=50)
m_KNneigh = KNeighborsClassifier(n_neighbors=numOfNeighs)

model4 = VotingClassifier(estimators=[ ('dtc', m_AdaBoost), ('rfc', m_gradBoost), ('etc', m_KNneigh) ], voting='soft', weights= mixingWeights )
model4.fit(feature_matrix, label_array)

estimator_models = [ ('dtc', m_DecTree), ('rfc', m_RandForest), ('etc', m_ExTree),('dtc', m_AdaBoost), ('rfc', m_gradBoost), ('etc', m_KNneigh),
                   ('MNB',model1)]
model5 = VotingClassifier(estimators=estimator_models, voting='hard')
model5.fit(feature_matrix, label_array)


model6 = GaussianNB()
model7 = BernoulliNB()

model6.fit(feature_matrix,label_array)
model7.fit(feature_matrix,label_array)


estimator_models = [ ('m1', model1), ('m6', model6), ('m7', model7)]
model8 = VotingClassifier(estimators=estimator_models, voting='soft')
model8.fit(feature_matrix, label_array)




# Fit the extra models
m_AdaBoost.fit(feature_matrix, label_array)
m_gradBoost.fit(feature_matrix, label_array)
m_KNneigh.fit(feature_matrix, label_array)

print '========================================================================================================================'
print 'Fitting of Models is Complete'
print '========================================================================================================================'

#========================================================================================================================
#========================================================================================================================
# Now we make a prediction on the testing data set
feature_matrix_test = nr.create_featureMatrix(testing_df,dictionary, size=testing_size)
label_array_test = nr.create_label_array(testing_df,size= testing_size)

# Calculate the results of all of the models
result1 = model1.predict(feature_matrix_test)
result2 = model2.predict(feature_matrix_test)

# Make Predictions with the second ensemble models


# Make Predictions with the third ensemble models
result3 = model3.predict(feature_matrix_test)
result3_1 = m_DecTree.predict(feature_matrix_test)
result3_2 = m_RandForest.predict(feature_matrix_test)
result3_3 = m_ExTree.predict(feature_matrix_test)

result4 = model4.predict(feature_matrix_test)
result4_1 = m_AdaBoost.predict(feature_matrix_test)
result4_2 = m_gradBoost.predict(feature_matrix_test)
result4_3 = m_KNneigh.predict(feature_matrix_test)

result5 = model5.predict(feature_matrix_test)

result6 = model6.predict(feature_matrix_test)
result7 = model7.predict(feature_matrix_test)
result8 = model8.predict(feature_matrix_test)

print '========================================================================================================================'
print 'Model Predictions on Test Set Complete'
print '========================================================================================================================'




cm1 = confusion_matrix(label_array_test,result1)
cm2 = confusion_matrix(label_array_test,result2)
cm3 = confusion_matrix(label_array_test,result3)
cm3_1 = confusion_matrix(label_array_test,result3_1)
cm3_2 = confusion_matrix(label_array_test,result3_2)
cm3_3 = confusion_matrix(label_array_test,result3_3)

cm4 = confusion_matrix(label_array_test,result4)
cm4_1 = confusion_matrix(label_array_test,result4_1)
cm4_2 = confusion_matrix(label_array_test,result4_2)
cm4_3 = confusion_matrix(label_array_test,result4_3)
cm5 = confusion_matrix(label_array_test,result5)

cm6 = confusion_matrix(label_array_test,result6)
cm7 = confusion_matrix(label_array_test,result7)
cm8 = confusion_matrix(label_array_test,result8)

print '======================================'
print 'Unormalized C1', cm1,'\n'
print 'Unormalized C2', cm2,'\n'
print 'Unormalized C3', cm3,'\n'
print 'Unormalized C3_DecTree', cm3_1,'\n'
print 'Unormalized C3_Random_Forest', cm3_2,'\n'
print 'Unormalized C3_Extra_Tree', cm3_3,'\n'
print 'Unormalized C4', cm4,'\n'
print 'Unormalized C4_AdaBoost', cm4_1,'\n'
print 'Unormalized C4_gradiantBoost', cm4_2,'\n'
print 'Unormalized C4_KNearestN',cm4_3,'\n'
print 'Unormalized C5_Voting',cm5,'\n'
print 'Unormalized C6_NB_Gauss',cm6,'\n'
print 'Unormalized C7 NB_Bernulli',cm7,'\n'
print 'Unormalized C8 NB_Ensemble',cm8,'\n'
print '======================================'

# Now we print off the Confusion matrix for the models
cm1 = cm1.astype('float') / cm1.sum(axis=1)[:, np.newaxis]
cm2 = cm2.astype('float') / cm2.sum(axis=1)[:, np.newaxis]
cm3 = cm3.astype('float') / cm3.sum(axis=1)[:, np.newaxis]
cm3_1 = cm3_1.astype('float') / cm3_1.sum(axis=1)[:, np.newaxis]
cm3_2 = cm3_2.astype('float') / cm3_2.sum(axis=1)[:, np.newaxis]
cm3_3 = cm3_3.astype('float') / cm3_3.sum(axis=1)[:, np.newaxis]
cm4 = cm4.astype('float') / cm4.sum(axis=1)[:, np.newaxis]
cm4_1 = cm4_1.astype('float') / cm4_1.sum(axis=1)[:, np.newaxis]
cm4_2 = cm4_2.astype('float') / cm4_2.sum(axis=1)[:, np.newaxis]
cm4_3 = cm4_3.astype('float') / cm4_3.sum(axis=1)[:, np.newaxis]
cm5 = cm5.astype('float') / cm5.sum(axis=1)[:, np.newaxis]
cm6 = cm6.astype('float') / cm6.sum(axis=1)[:, np.newaxis]
cm7 = cm7.astype('float') / cm7.sum(axis=1)[:, np.newaxis]
cm8 = cm8.astype('float') / cm8.sum(axis=1)[:, np.newaxis]


print '======================================'
print 'Normalized C1', cm1,'\n'
print 'Normalized C2', cm2,'\n'
print 'Normalized C3', cm3,'\n'
print 'Normalized C3_DecTree', cm3_1,'\n'
print 'Normalized C3_Random_Forest', cm3_2,'\n'
print 'Normalized C3_Extra_Tree', cm3_3,'\n'
print 'Normalized C4', cm4,'\n'
print 'Normalized C4_AdaBoost', cm4_1,'\n'
print 'Normalized C4_gradiantBoost', cm4_2,'\n'
print 'Normalized C4_KNearestN',cm4_3,'\n'
print 'Normalized C5 Voting',cm5,'\n'
print 'Unormalized C6_NB_Gauss',cm6,'\n'
print 'Unormalized C7 NB_Bernulli',cm7,'\n'
print 'Unormalized C8 NB_Ensemble',cm8,'\n'
print '======================================'

# Now we should write out the results of the training

directory = 'model_files/'

names = ['NB_MultiNomial.pkl','SVC.pkl','Tree_vote.pkl',
         'DecTree.pkl','RandFor.pkl','Extree.pkl','otherM_vote.pkl',
         'AdaB.pkl','gradB.pkl','KNneigh.pkl','tree_and_otherM_vote.pkl','NB_Gauss.pkl','NB_Bern.pkl','NB_vote.pkl']

cm_names = ['NB_MultiNomial_cm.dat','SVC_cm.dat','Tree_vote_cm.dat',
         'DecTree_cm.dat','RandFor_cm.dat','Extree_cm.dat','otherM_vote_cm.dat',
         'AdaB_cm.dat','gradB_cm.dat','KNneigh_cm.dat','tree_and_otherM_vote_cm.dat','NB_Gauss_cm.dat','NB_Bern_cm.dat','NB_vote_cm.dat']

model_array = [model1,model2,model3,m_DecTree,m_RandForest,m_ExTree,model4,
                m_AdaBoost,m_gradBoost,m_KNneigh,model5,model6,model7,model8]  
cm_array = [cm1,cm2,cm3,cm3_1,cm3_2,cm3_3,cm4,cm4_1,cm4_2,cm4_3,cm5,cm6,cm7,cm8]

for k in range(len(names)):
    name = directory+'dict_size_'+str(dictionary_size)+'_'+'dataSet_'+str(len(real_and_fake_df))+'_'+names[k]
    with open(name, 'wb') as fid:
        cPickle.dump(model_array[k], fid)

# Write all cm to file
for k in range(len(cm_names)):
    cm_name = directory+'dict_size_'+str(dictionary_size)+'_'+'dataSet_'+str(len(real_and_fake_df))+'_'+cm_names[k]
    f=open(cm_name,'w')
    pickle.dump(cm_array[k], f)
    f.close()
    
    
print '====================================================================================='  
print 'cm arrays saved to file'
print 'pkl files saved to file'
print 'Program Complete'
print '====================================================================================='

Training Set Number:  400
Testing Set Number:  100
The Size of the dictionary 2515
Top Dictionary Words:  [('the', 48), ('trump', 23), ('in', 15), ('to', 14), ('us', 14), ('clinton', 13), ('hillary', 11), ('neil', 11), ('for', 10), ('young', 10)]

array of labels size:  400
feature matrix:  400
Feature Matrix computation completed
Fitting will now take place
Fitting of Models is Complete
Model Predictions on Test Set Complete
Unormalized C1 [[40  6]
 [19 35]] 

Unormalized C2 [[41  5]
 [20 34]] 

Unormalized C3 [[45  1]
 [44 10]] 

Unormalized C3_DecTree [[45  1]
 [42 12]] 

Unormalized C3_Random_Forest [[46  0]
 [45  9]] 

Unormalized C3_Extra_Tree [[45  1]
 [43 11]] 

Unormalized C4 [[45  1]
 [46  8]] 

Unormalized C4_AdaBoost [[42  4]
 [32 22]] 

Unormalized C4_gradiantBoost [[42  4]
 [33 21]] 

Unormalized C4_KNearestN [[45  1]
 [51  3]] 

Unormalized C5_Voting [[43  3]
 [38 16]] 

Unormalized C6_NB_Gauss [[33 13]
 [10 44]] 

Unormalized C7 NB_Bernulli [[42  4]
 [24 30]] 

Unormali